# GENERATE PASS THRU DEMAND

This module takes the pass thru demand from people dataframe and creates xml file for pass thru demand. 

```python
def create_sumo_demand_passthru(people):
```

> - **people:** *DataFrame* - is the people demand for the 


In [1]:
from __future__ import print_function
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, SubElement, Comment
from lxml import etree
from copy import copy
import os
import sys
import inspect
from xml.dom import minidom
import pandas as pd
import numpy as np
import random
sys.path.insert(0, os.path.abspath('../Generate_SUMO_demand/'))
import DFW_gen_flow as gf

In [2]:
Date = '2018-5-13' 

In [3]:
folder = "../Example_Files/Demand_CSV"
selected = pd.read_csv("selected_demand.csv")
level = selected.loc[selected['Date'] == Date]['Volume_Category'].item()
file_name = Date + '.' + level +".people.csv"
people = pd.read_csv(os.path.join(folder,file_name),index_col=[0])

people.head()

,Arrive_A_people,Arrive_B_people,Arrive_C_people,Arrive_D_people,Arrive_E_people,Depart_A_people,Depart_B_people,Depart_C_people,Depart_D_people,Depart_E_people,Depart_total,Arrival_total,pass_thru,Total
0,120,65,126,65,12,120,67,128,65,12,392,388,44.0,780
1,70,39,77,31,7,70,41,79,31,7,228,224,24.0,452
2,50,29,56,26,5,51,29,58,26,5,169,166,32.0,335
3,58,34,63,31,5,58,34,65,31,5,193,191,16.0,384
4,31,19,34,19,5,32,19,36,19,7,113,108,32.0,221


In [19]:
def create_sumo_demand_passthru(people):
    
    starts = ['South_1', 'South_Plaza', 'North_Plaza', 'North_1']
    ends = ['South_Exit', 'North_Exit']
    start_weights = [.225,.225,.275,.275]
    routes = Element('routes')
    routes.set('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
    routes.set('xsi:noNamespaceSchemaLocation', 'http://sumo.dlr.de/xsd/routes_file.xsd')
    people['seconds'] = np.array(people.index) * 30 * 60
    count = 1

    
    for t,numberOfCars in enumerate(people['pass_thru']):
        
        for i in range(int(numberOfCars)):
            time = people['seconds'][t] + round(np.random.uniform(0,1800))
            start = np.random.choice(starts,p=start_weights)
            if start[0] == 'S':
                end = np.random.choice(ends,p=[.1,.9])
            else:
                end = np.random.choice(ends,p=[.9,.1])
            
            trip = Element('trip')
            trip.set('id','pass_' + str(count))
            trip.set('type', 'passenger')
            trip.set('color', "#bb0000")
            trip.set('depart',str(time))
            trip.set('from',start)
            trip.set('to',end)
            trip.set('departSpeed', "max")
            trip.set('departLane', "best")

            count+=1
            routes.append(trip)


    routes[:] = sorted(routes, key=lambda child: (child.tag,float(child.get('depart'))))

    file_name = Date + "." + level+ ".passthru.xml"
    folder = "../Example_Files/TempDemandXML"
    print("Saving to xml: ", file_name)
        
    with open(os.path.join(folder,file_name), 'wb') as f:
        f.write(minidom.parseString(ET.tostring(routes)).toprettyxml(encoding="utf-8"))
    

In [20]:
create_sumo_demand_passthru(people)

Saving to xml:  2018-5-13.high.passthru.xml
